In [1]:

import dask
import dask.array as da
import torch
import numpy as np
tensor = torch.randn(501,300,20000)
chunks_size = (50,300,20000)
dask_array = da.from_array(tensor.numpy(), chunks=chunks_size)  
print(dask_array.shape)
import torch.nn as nn
model = nn.Sequential(nn.Flatten(),nn.Linear(in_features=300*200*100, out_features=10))  
def inference(chunk, model):
 
    model.eval()


    expected_shape = (chunk.shape[0], 300, 20000)
    assert chunk.shape == expected_shape, f"Invalid chunk shape: {chunk.shape}"

    with torch.no_grad():
        chunk_tensor = torch.from_numpy(chunk)
        chunk_flat = chunk_tensor.view(chunk_tensor.size(0), -1)
        output=model(chunk_flat)
        
    output=output.detach().numpy()
    output = output[:,np.newaxis]
    return output

result = dask_array.map_blocks(inference,model=model,dtype=np.float32,chunks = chunks_size)
result_array = result.compute()
#


(501, 300, 20000)


In [2]:
result_array.shape

(501, 1, 10)

In [3]:
result_serial = model(tensor)
result_serial.shape

torch.Size([501, 10])

In [4]:
!pip install h5py


Defaulting to user installation because normal site-packages is not writeable


In [5]:
import h5py
import torch

# Suppose your tensor is `tensor` (batch, channel, height, width)
tensor = torch.randn(500,60, 224, 224)

# Create an HDF5 file
with h5py.File('tensor_data.h5', 'w') as hf:
    # Create a dataset within the HDF5 file
    hf.create_dataset('data', data=tensor.numpy())


In [2]:
import h5py

# Ouverture du fichier HDF5 en mode lecture
with h5py.File('tensor_data.h5', 'r') as f:
    # Accéder au jeu de données
    dataset = f["./"]


In [10]:
dataset

<Closed HDF5 group>

In [1]:
%load_ext autoreload
%autoreload 2
import sys
import torch
sys.path.append('../src')
sys.path.append('../util')
from torch import sparse
from abstractModule import AbstractLinear as AL
from abstract import abstractTensor as AT

In [3]:
tensor_to_abstract = torch.randn(3,112,112)
abstract_tensor = AT(tensor_to_abstract,alpha=0.03*torch.ones(3*112*112)).abstract_tensor()
abstract_tensor.shape


torch.Size([37634, 3, 112, 112])

In [5]:

abstract_tensor = abstract_tensor.to_sparse()

In [6]:
abstract_tensor


tensor(indices=tensor([[    0,     0,     0,  ..., 12542, 12543, 12544],
                       [    0,     0,     0,  ...,     0,     0,     0],
                       [    0,     0,     0,  ...,   111,   111,   111],
                       [    0,     1,     2,  ...,   109,   110,   111]]),
       values=tensor([0.7535, 0.1596, 0.0033,  ..., 0.0300, 0.0300, 0.0300]),
       size=(12546, 1, 112, 112), nnz=25088, layout=torch.sparse_coo)

In [10]:
indices = torch.arange(0,20)
indices = torch.tensor(indices)
result = abstract_tensor.index_select(0, indices)

print(result.shape)
result = result.to_sparse()

abstract_tensor.index_copy_(dim =0, index = indices, source = result)

torch.Size([20, 3, 112, 112])


/tmp/ipykernel_124719/2735095733.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  indices = torch.tensor(indices)


NotImplementedError: Could not run 'aten::index_copy_' with arguments from the 'SparseCPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::index_copy_' is only available for these backends: [CPU, CUDA, HIP, MPS, IPU, XPU, HPU, VE, MTIA, PrivateUse1, PrivateUse2, PrivateUse3, Meta, FPGA, ORT, Vulkan, Metal, QuantizedCPU, QuantizedCUDA, QuantizedHIP, QuantizedMPS, QuantizedIPU, QuantizedXPU, QuantizedHPU, QuantizedVE, QuantizedMTIA, QuantizedPrivateUse1, QuantizedPrivateUse2, QuantizedPrivateUse3, QuantizedMeta, CustomRNGKeyId, MkldnnCPU, SparseCsrCPU, SparseCsrCUDA, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradHIP, Autograd

In [8]:
import torch

# Créer un tenseur creux (sparse tensor) original
sparse_tensor = torch.sparse_coo_tensor(indices=[[0, 1, 1],
                                                 [1, 0, 1]],
                                        values=[1, 2, 3],
                                        size=(2, 2))

# Afficher le tenseur creux original
print("Tenseur creux original:")
print(sparse_tensor)

# Sélectionner un sous-ensemble de valeurs avec index_select
selected_indices = torch.tensor([0, 2])
selected_values = torch.sparse.index_select(sparse_tensor, dim=0, index=selected_indices)

# Afficher les indices et les valeurs sélectionnés
print("\nIndices sélectionnés:")
print(selected_indices)
print("Valeurs sélectionnées:")
print(selected_values)

# Créer de nouvelles valeurs à remplacer
new_values = torch.tensor([10, 20])

# Reconstruire le tenseur creux avec les nouvelles valeurs
# Utilisez le constructeur sparse_coo_tensor pour reconstruire le tenseur
# En passant les nouveaux indices sélectionnés et les nouvelles valeurs
sparse_tensor = torch.sparse_coo_tensor(indices=selected_indices.numpy(),  # Convertir en tableau numpy
                                        values=new_values,
                                        size=(2, 2))

# Afficher le tenseur creux modifié
print("\nTenseur creux modifié:")
print(sparse_tensor)


Tenseur creux original:
tensor(indices=tensor([[0, 1, 1],
                       [1, 0, 1]]),
       values=tensor([1, 2, 3]),
       size=(2, 2), nnz=3, layout=torch.sparse_coo)


AttributeError: module 'torch.sparse' has no attribute 'index_select'

In [8]:
device = torch.device('cpu')
abstract_tensor.to(device)

tensor(indices=tensor([[    0,     0,     0,  ..., 12542, 12543, 12544],
                       [    0,     0,     0,  ...,     0,     0,     0],
                       [    0,     0,     0,  ...,   111,   111,   111],
                       [    0,     1,     2,  ...,   109,   110,   111]]),
       values=tensor([0.7535, 0.1596, 0.0033,  ..., 0.0300, 0.0300, 0.0300]),
       size=(12546, 1, 112, 112), nnz=25088, layout=torch.sparse_coo)

In [9]:
indices


tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19])

In [10]:
result = result.to_dense()
print(result[0].shape)

result = torch.randn(20,3,224,224)

result = result.to_sparse()
abstract_tensor.index_put_([indices],result)

torch.Size([1, 112, 112])


NotImplementedError: Could not run 'aten::_index_put_impl_' with arguments from the 'SparseCPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::_index_put_impl_' is only available for these backends: [CPU, CUDA, Meta, QuantizedCPU, QuantizedCUDA, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradHIP, AutogradXLA, AutogradMPS, AutogradIPU, AutogradXPU, AutogradHPU, AutogradVE, AutogradLazy, AutogradMTIA, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, AutogradMeta, AutogradNestedTensor, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrap

In [20]:
abstract_tensor.indices()[1,:,:,:]

NameError: name 'abstract_tensor' is not defined

In [19]:
import h5py
import torch
import numpy as np

indices = np.arange(0,100,1) 


with h5py.File('tensor_data.h5', 'r') as hf:
    
    total_size = hf['data'].shape[0]
    print(total_size)
    
    
    subset_tensors = []
    
    
    chunk_size = 100  
    
    
    for start_index in range(0, total_size, chunk_size):
        end_index = min(start_index + chunk_size, total_size)
        

        data_chunk = hf['data'][start_index:end_index]
        print(data_chunk.shape)
      
        chunk_subset = torch.tensor(data_chunk[indices])
        print(chunk_subset.shape)
        

        subset_tensors.append(chunk_subset)
        

subset_tensor = torch.cat(subset_tensors,dim =0)
subset_tensor.squeeze(0)

print("Taille du subset_tensor:", subset_tensor.size())


print(subset_tensor)


500
(100, 60, 224, 224)
torch.Size([100, 60, 224, 224])
(100, 60, 224, 224)
torch.Size([100, 60, 224, 224])
(100, 60, 224, 224)
torch.Size([100, 60, 224, 224])
(100, 60, 224, 224)
torch.Size([100, 60, 224, 224])
(100, 60, 224, 224)
torch.Size([100, 60, 224, 224])
Taille du subset_tensor: torch.Size([500, 60, 224, 224])
tensor([[[[ 4.2692e-01,  1.3476e-01, -1.0956e+00,  ..., -8.5284e-01,
            5.0291e-01,  5.5153e-01],
          [-3.5254e-01,  1.1187e+00, -7.0639e-01,  ...,  1.2570e+00,
            9.8905e-01, -5.1879e-01],
          [ 4.1082e-01,  2.0111e-01,  7.4048e-01,  ..., -1.5556e+00,
            1.3142e-02, -6.3195e-01],
          ...,
          [-2.5921e+00,  6.3167e-01,  3.7550e-01,  ...,  6.4648e-01,
           -5.2637e-01, -1.8517e-01],
          [ 1.4282e+00,  4.3522e-01, -1.3143e+00,  ..., -1.2271e+00,
           -4.0522e-01, -1.0492e+00],
          [ 1.5851e-01,  6.9736e-01, -1.2156e+00,  ..., -1.1821e+00,
            8.0591e-01,  1.6482e-01]],

         [[-1.6304e-

In [123]:
%load_ext autoreload
%autoreload 2
import sys
import torch
sys.path.append('../src')
sys.path.append('../util')
from abstractNN import AbstractNN as NN
from abstract import abstractTensor as AT

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [180]:
tensor_to_abstract = torch.randn(1)
abstract_tensor = AT(tensor_to_abstract,alpha=0.03*torch.ones(1)).abstract_tensor()

In [166]:
abstract_tensor

tensor([[-0.0871],
        [ 0.0300],
        [ 0.0000]])

In [218]:
import torch.nn as nn
lin = nn.Sequential(nn.Flatten(),nn.Linear(in_features=1,out_features=2))
lin[1].weight.data= torch.tensor([[1.],[-1.]])
lin[1].bias.data = torch.zeros_like(lin[1].bias.data)

In [168]:
torch.topk(lin[1].weight.data.flatten(), k = 10).values
torch.topk(lin[1].weight.data.flatten(), k = 10)


RuntimeError: selected index k out of range

In [198]:
lin_eps

torch.return_types.topk(
values=tensor([1., 1.]),
indices=tensor([0, 1]))

In [223]:
lin_eps_weight =torch.tensor([])
lin_eps_weight.indices = torch.topk(lin[1].weight.data.flatten(),2).indices

print(lin_eps.indices)
lin_eps_weight.values = torch.ones_like(lin_eps_weight.indices)
print(lin_eps_weight.values)


lin_eps_bias = torch.tensor([])
lin_eps_bias.indices = torch.topk(lin[1].bias.data,2).indices
lin_eps_bias.values = torch.ones_like(lin_eps_bias.indices)
x,x_min,x_max,x_true = NN.abstract_linear(lin, 
                                          abstract_tensor,
                                          abstract_tensor[0].unsqueeze(0),
                                          lin_eps_weight,
                                          lin_eps_bias)

tensor([0, 1])
tensor([1, 1])
torch.Size([2, 2])
torch.Size([2, 2])


In [224]:
x.shape

torch.Size([7, 2])

In [225]:
abstract_tensor

tensor([[-3.0937],
        [ 0.0300],
        [ 0.0000]])

In [226]:
x

tensor([[-3.0937,  3.0937],
        [ 0.0300, -0.0300],
        [-3.0937,  0.0000],
        [ 0.0000, -3.0937],
        [ 1.0000,  0.0000],
        [ 0.0000,  1.0000],
        [ 0.0000,  0.0000]])

In [252]:
conv = nn.Conv2d(2,2,2)
conv.weight.data = torch.zeros_like(conv.weight.data)
conv.bias.data = torch.zeros_like(conv.bias.data)

In [253]:
tensor_to_abstract = torch.randn(2,2,2)
abstract_tensor = AT(tensor_to_abstract,alpha=0.03*torch.ones(2*1*2)).abstract_tensor()

conv_eps_weight =torch.tensor([])
conv_eps_weight.indices = torch.topk(conv.weight.data.flatten(),k =2).indices

print(conv_eps_weight.indices)
conv_eps_weight.values = torch.ones_like(conv_eps_weight.indices)
print(conv_eps_weight.values)


conv_eps_bias = torch.tensor([])
conv_eps_bias.indices = torch.topk(conv.bias.data,2).indices
conv_eps_bias.values = torch.ones_like(conv_eps_bias.indices)
x,x_min,x_max,x_true = NN.abstract_conv2D(conv, 
                                          abstract_tensor,
                                          abstract_tensor[0].unsqueeze(0),
                                          conv_eps_weight,
                                          conv_eps_bias)

tensor([10, 11])
tensor([1, 1])
torch.Size([2, 2, 1, 1])
torch.Size([2, 2, 1, 1])


In [246]:
abstract_tensor[0]

tensor([[[-0.5851, -0.1173],
         [ 0.5130,  0.1623]],

        [[ 1.5964,  0.3390],
         [-0.8472,  0.4738]]])

In [254]:
x

tensor([[[[ 0.0000]],

         [[ 0.0000]]],


        [[[ 0.0000]],

         [[ 0.0000]]],


        [[[ 0.0000]],

         [[ 0.0000]]],


        [[[ 0.0000]],

         [[ 0.0000]]],


        [[[ 0.0000]],

         [[ 0.0000]]],


        [[[ 0.0000]],

         [[-0.1608]]],


        [[[ 0.0000]],

         [[-0.4494]]],


        [[[ 1.0000]],

         [[ 0.0000]]],


        [[[ 0.0000]],

         [[ 1.0000]]],


        [[[ 0.0000]],

         [[ 0.0000]]]])